In [1]:
import tweepy
import pandas as pd
from datetime import date
import time
import warnings
import os
from langdetect import detect
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\armag\AppData\Roaming\Python\Python37\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
consumer_key = '7JlnkfQYVgs1x1mnMvm6nkOCc'
consumer_secret = 'O99q9aa2kgKOo4ps1MNy4uJQx06VLOPjzKN6SxrJixNi1jNPgG'
access_token = '520058445-WwG5wY1UIoMThSldZ0JY3DoKyPcHXv3X1pNxr86h'
access_token_secret = '4OUVn4kTqKvDXEAF4YYiNd0wLZqRT5Ti7TPUJJvxGWou8'
selected_language = 'ar'
location = r'YOUR_LOCATION' ######LOCATION THAT YOU WANT TO SAVE CSV FILE

In [3]:
#connecting to API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
if not os.path.exists(location):
    df = pd.DataFrame()
    df.to_csv(location, encoding='utf-8-sig')

In [5]:
#Taking all the WOEID codes of Countries
woeid_list = []
for val in api.trends_available():
    if val['placeType']['name'] == 'Country':#if you want to extract more tweet change 'Country' to 'Town', but execution time will increase.
        woeid_list.append(val['woeid'])

In [6]:
len(woeid_list)

62

In [7]:
all_tweet_lists = []
count_down = 0

In [8]:
#Collecting all the trends and appending to the list
for woeid in woeid_list:
    trends = api.trends_place(woeid)
    all_tweet_lists.append(trends)

In [9]:
twitter_trends = {
    'Name':[]
}

In [10]:
len(all_tweet_lists)

62

In [12]:
#Appending arabic tweets to the twitter_trends['Name']
count=0
for tweet in all_tweet_lists:
    for value in tweet:
        for trend in value['trends']:
            try:
                if detect(trend['name']) == selected_language: #detecting language
                    twitter_trends['Name'].append(trend['name'])
            except:
                print('This text is passed because of NULL value.')

    count += 1
    print(count/len(all_tweet_lists)*100)

1.6129032258064515
3.225806451612903
4.838709677419355
6.451612903225806
8.064516129032258
9.67741935483871
11.29032258064516
12.903225806451612
14.516129032258066
16.129032258064516
17.741935483870968
19.35483870967742
20.967741935483872
22.58064516129032
24.193548387096776
25.806451612903224
27.419354838709676
29.03225806451613
30.64516129032258
32.25806451612903
33.87096774193548
35.483870967741936
37.096774193548384
38.70967741935484
40.32258064516129
41.935483870967744
43.54838709677419
45.16129032258064
46.774193548387096
48.38709677419355
50.0
51.61290322580645
53.2258064516129
54.83870967741935
56.451612903225815
58.06451612903226
59.67741935483871
61.29032258064516
62.903225806451616
64.51612903225806
66.12903225806451
67.74193548387096
69.35483870967742
70.96774193548387
72.58064516129032
74.19354838709677
75.80645161290323
This text is passed because of NULL value.
77.41935483870968
79.03225806451613
80.64516129032258
82.25806451612904
83.87096774193549
85.48387096774194
87.

In [13]:
len(twitter_trends['Name'])

230

In [14]:
limit_per_trend = 900//len(twitter_trends['Name'])
limit_per_trend

3

In [15]:
tweets_output = {
    'trend_name': [],
    'trend_tweet': []
}
temp = []

In [ ]:
for trend in twitter_trends['Name']:     
    for tweet in tweepy.Cursor(api.search, q=trend, count=100000, lang=selected_language, since=date.today(),result_type='popular').items(limit_per_trend):
        tweets_output['trend_name'].append(trend)
        temp.append(tweet.text)

In [ ]:
for a in temp:
    a = a.replace('\n', "")
    tweets_output['trend_tweet'].append(a)

In [ ]:
data = pd.read_csv(location)
df = pd.DataFrame(tweets_output)

In [ ]:
final = pd.concat([data, df], ignore_index=True).drop_duplicates().reset_index(drop=True)

In [ ]:
final

In [ ]:
final.to_csv(location, mode='a', encoding='utf-8-sig', index=False)